In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv
/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv


In [115]:
train_data = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding='ISO-8859-1')
test_data = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_test.csv',encoding='ISO-8859-1')

for i in range(len(train_data)):
    if (train_data.Sentiment.iloc[i]) == 'Extremely Positive':
        train_data.Sentiment.iloc[i] = 'Positive'
    if (train_data.Sentiment.iloc[i]) == 'Extremely Negative':
        train_data.Sentiment.iloc[i] = 'Negative'

for i in range(len(test_data)):
    if (test_data.Sentiment.iloc[i]) == 'Extremely Positive':
        test_data.Sentiment.iloc[i] = 'Positive'
    if (test_data.Sentiment.iloc[i]) == 'Extremely Negative':
        test_data.Sentiment.iloc[i] = 'Negative'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [116]:
import spacy

# Create an empty model
nlp = spacy.blank("en")

# Create the TextCategorizer with exclusive classes and "bow" architecture
textcat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Add the TextCategorizer to the empty model
nlp.add_pipe(textcat)

In [118]:
# Add labels to text classifier
textcat.add_label("Negative")
textcat.add_label("Positive")
textcat.add_label("Neutral")

1

In [119]:
train_tweets = train_data.OriginalTweet.values
test_tweets = test_data.OriginalTweet.values

train_labels = [{'cats': {'Negative': label == 'Negative',
                          'Positive' : label == 'Positive',
                          'Neutral' : label == 'Neutral'}} 
                for label in train_data['Sentiment']]
test_labels = [{'cats': {'Negative': label == 'Negative',
                          'Positive' : label == 'Positive',
                          'Neutral' : label == 'Neutral'}} 
                for label in test_data['Sentiment']]

train_data = list(zip(train_tweets, train_labels))
test_data = list(zip(test_tweets, test_labels))

In [120]:
from spacy.util import minibatch
import random

def train(model, train_data, optimizer):
    losses = {}
    random.seed(1)
    random.shuffle(train_data)
    
    batches = minibatch(train_data, size=8)
    for batch in batches:
        tweets, labels = zip(*batch)
        
        # Update model with texts and labels
        model.update(tweets, labels, sgd=optimizer, losses=losses)
    return losses

In [121]:
def predict(nlp, tweets): 
    # Use the model's tokenizer to tokenize each input text
    docs = [nlp.tokenizer(tweet) for tweet in tweets]
    
    # Use textcat to get the scores for each doc
    textcat = nlp.get_pipe('textcat')
    scores, _ = textcat.predict(docs)
    
    # From the scores, find the class with the highest score/probability
    predicted_class = scores.argmax(axis=1)
    
    return predicted_class

In [122]:
# Fix seed for reproducibility
spacy.util.fix_random_seed(1)
random.seed(1)

# This may take a while to run!
optimizer = nlp.begin_training()
losses = train(nlp, train_data, optimizer)
print(losses['textcat'])

33.67964443413075


In [123]:
texts = test_tweets[:30]
predictions = predict(nlp, texts)

for p, t in zip(predictions, texts):
    print(f"{textcat.labels[p]}: {t} \n")

Positive: TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1 

Positive: When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY 

Positive: Find out how you can protect yourself and loved ones from #coronavirus. ? 






#Election2020 #CDC https://t.co/29isZOewxu 

Negative: #toiletpaper #dunnypaper #coronavirus #coronavirusaustralia #CoronaVirusUpdate #Covid_19 #9News  #Corvid19 #7NewsMelb #dunnypapergate #Costco    One week everyone buying baby milk powder the next everyone buying up toilet paper. https://t.co/ScZryVvsIh 

Neutral: Do you remember the last time you paid $2.99 a gallon for regular gas in Los Angeles?Prices at the pump are going down. A look at how the #coro

In [125]:
docs=[nlp.tokenizer(tweet) for tweet in test_tweets]
textcat = nlp.get_pipe('textcat')
scores, _ = textcat.predict(docs)
predicted_class = scores.argmax(axis=1)

array([1, 1, 1, ..., 2, 0, 1])

In [126]:
def evaluate(model, tweets, labels):
    
    # Get predictions from textcat model (using your predict method)
    predicted_class = predict(model, tweets)
    
    # From labels, get the true class as a list of integers (POSITIVE -> 1, NEGATIVE -> 0)
    true_class = []
    for each in labels:
        if (each['cats']['Negative']):
            true_class.append(0)
        if (each['cats']['Positive']):
            true_class.append(1)
        if (each['cats']['Neutral']):
            true_class.append(2)
    
    # A boolean or int array indicating correct predictions
    correct_predictions = predicted_class == true_class
    
    # The accuracy, number of correct predictions divided by all predictions
    accuracy = correct_predictions.mean()
    
    return accuracy

In [131]:
# This may take a while to run!
n_iters = 5
for i in range(1,n_iters+1):
    losses = train(nlp, train_data, optimizer)
    accuracy = evaluate(nlp, test_tweets, test_labels)
    print(f"Itération N°{i} \t Loss: {losses['textcat']:.3f} \t Accuracy: {accuracy:.3f}")

Itération N°1 	 Loss: 1.735 	 Accuracy: 0.767
Itération N°2 	 Loss: 1.566 	 Accuracy: 0.769


KeyboardInterrupt: 